In [632]:
import pandas as pd
import numpy as np
import random as rm
import re

In [633]:
sharks = pd.read_csv('../Data/attacks.csv',encoding='utf-8',encoding_errors= 'ignore')

In [634]:
sharks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [635]:
#Eliminamos los duplicados que tras cierta exploracion coinciden con la mayoria de valores nulos
sharks = sharks.drop_duplicates() 

In [636]:
pd.set_option('display.max_columns',None)
sharks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [637]:
sharks['Case Number'].value_counts() #podemos ver que como mucho, salvando los 0s que no dejan de ser nulos, solo hay duplicados por pares

0               8
1920.00.00.b    2
1966.12.26      2
2014.08.02      2
1990.05.10      2
               ..
1999.09.05      1
1999.09.10      1
1999.09.16      1
1999.09.18      1
xx              1
Name: Case Number, Length: 6287, dtype: int64

In [638]:
sharks['Case Number'].fillna('0',inplace=True)

In [639]:
sharks['Case Number'].value_counts()

0               10
2009.12.18       2
1920.00.00.b     2
2013.10.05       2
1983.06.15       2
                ..
1999.09.05       1
1999.09.10       1
1999.09.16       1
1999.09.18       1
xx               1
Name: Case Number, Length: 6287, dtype: int64

In [640]:
def Fix_Points_ID(x):

  '''
  Función que quita los puntos del ID para dejar un número realmente identificatorio del suceso
  Los duplicados serán corregidos en otra función
  '''
  x = str(x)     #Me quito el problema de que algunos sean float(presuntamente los 0s que tambien solucionaremos mas adelante)
  if '.' in x:
         
         p = x.split('.')
         x = ''
         for n in range(len(p)):
            x +=p[n]
         x +='0'  #Sera el default para los duplicados que cambiaré en otra función
  return x
def Fix_Letters_ID(x):

  '''
  Función que quita las letras del ID para dejar un número realmente identificatorio del suceso
  Los duplicados serán corregidos en otra función
  '''

  if 'a' in x or 'b' in x:
         if 'a' in x:
             p = x.split('a')
             x = ''
             for n in range(len(p)):
                 x +=p[n]
         elif 'b' in x:
             p = x.split('b')
             x = ''
             for n in range(len(p)):
                 x +=p[n]    
         
  return x

sharks['Case Number'] = sharks['Case Number'].apply(Fix_Points_ID)
sharks['Case Number'] = sharks['Case Number'].apply(Fix_Letters_ID)
sharks.head(10)     
        

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,201806250,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,201806180,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,201806090,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,201806080,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,201806040,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
5,201806030,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,NaN,NaN
6,201806030,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0,NaN,NaN
7,201805270,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0,NaN,NaN
8,201805260,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,NaN,NaN
9,201805260,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,NaN,NaN


In [641]:
no_dupl = []
p = ''
for i in range(len(sharks['Case Number'])): #Eliminamos duplicados que no sean 0s
    if sharks['Case Number'].iloc[i] != '0':
        if sharks['Case Number'].iloc[i] in no_dupl and sharks['Case Number'].iloc[i] != '0':
            p = sharks['Case Number'].iloc[i][:-1]
            p +='1'
            if p in no_dupl:
                p = sharks['Case Number'].iloc[i][:-1]
                p +='2'
                no_dupl.append(p)
            else:
                no_dupl.append(p)
        else:
            no_dupl.append(sharks['Case Number'].iloc[i])
    else:
        no_dupl.append(sharks['Case Number'].iloc[i])
sharks['Case Number'] = no_dupl
sharks['Case Number'].value_counts()


0            10
201806250     1
195705000     1
195702020     1
195702050     1
             ..
199909180     1
199909240     1
199909290     1
199910010     1
xx            1
Name: Case Number, Length: 6303, dtype: int64

In [642]:
#Elimino la fila con valor xx que está vacía y no aporta
sharks.drop(sharks.loc[sharks['Case Number']=='xx'].index,inplace=True)

In [643]:
#Renombro la columna por Attack ID que me parece mejor y más explicativa
nombres_col=[]
nombres_col = list(sharks.columns)
nombres_col[0] = 'Attack ID'
sharks.columns = nombres_col
sharks.head()

#No resuelvo aun los nulos porque la idea es introducir un numero del 0 al que corresponda a los mismos
#Como no se cuantas filas borraré no tiene mucho sentido hacerlo aun

,Attack ID,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,201806250,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,201806180,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,201806090,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,201806080,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,201806040,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [644]:
sharks['Date'].value_counts()

1957                          11
1942                           9
1956                           8
1958                           7
1950                           7
                              ..
01-Dec-1994                    1
Reported      10-Dec-1994      1
11-Dec-1994                    1
13-Dec-1994                    1
1845-1853                      1
Name: Date, Length: 5433, dtype: int64

In [645]:
sharks.loc[sharks['Date']=='1957']

,Attack ID,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
4211,19570000k0,1957,1957.0,Invalid,SRI LANKA,Western Province,Colombo Harbor,Fishing,F. L. Fernando,M,NaN,"FATAL, but shark involvement not confirmed",NaN,NaN,Shark involvement prior to death was not confi...,R. I. DeSilva,1957.00.00.k-Fernando.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.k,1957.00.00.k,2092.0,NaN,NaN
4212,19570000j0,1957,1957.0,Unprovoked,USA,Florida,"Fort Pierce, St. Lucie County",Bathing,R. Nauth,NaN,NaN,Injured by shark,N,NaN,1.5 m [5'] shark,R.F. Hutton,1957.00.00.j-NV-Nauth.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.j,1957.00.00.j,2091.0,NaN,NaN
4213,19570000i0,1957,1957.0,Invalid,CUBA,Havana Province,Cojimar,NaN,an infant,NaN,2 to 3 months,"FATAL, but was it an accident or infanticide?",NaN,NaN,3.7 m [12'] shark,"F. Poli, p.23-24",1957.00.00.i-baby,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.i,1957.00.00.i,2090.0,NaN,NaN
4214,19570000h0,1957,1957.0,Boat,CUBA,Havana Province,Cojimar,Fishing,"boat, occupant: Portuondo",NaN,NaN,"No injury to occupant, shark stuck boat",N,NaN,3.7 m [12'] shark,"F. Poli, pp.21-24",1957.00.00.h-boat-Portuondo.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.h,1957.00.00.h,2089.0,NaN,NaN
4215,19570000g0,1957,1957.0,Unprovoked,CUBA,Havana Province,Havana,Fishing for sharks,William Bolster,M,18,"FATAL, became entangled in fishing line and pu...",Y,NaN,NaN,"F. Poli, p.9",1957.00.00.g-Bolster.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.g,1957.00.00.g,2088.0,NaN,NaN
4216,19570000f0,1957,1957.0,Unprovoked,USA,Florida,"Miami, Miami-Dade County",Helmet diving in Miami Seaquarium,Jim Kline,M,NaN,"Shark struck helmet, no injury",N,NaN,Bull shark,"R. Carras, H.D. Baldridge, p.181",1957.00.00.f-Kline.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.f,1957.00.00.f,2087.0,NaN,NaN
4217,19570000e0,1957,1957.0,Unprovoked,FIJI,Kadavu,Great Astrolabe Reef,Fishing,Semesa Vasu,M,30,Right foot lacerated,N,NaN,NaN,S.B. Brown,1957.00.00.e-Vasu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.e,1957.00.00.e,2086.0,NaN,NaN
4218,19570000d0,1957,1957.0,Unprovoked,IRAN,Karun River,"near Band Misan in Shustar, 420 km from the sea",NaN,Mr. Paniry,M,NaN,Survived,N,NaN,Bull shark,B. Coad,1957.00.00.d-Paniry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.d,1957.00.00.d,2085.0,NaN,NaN
4219,195700000,1957,1957.0,Unprovoked,PAPUA NEW GUINEA,"Abau Subdistrict,Central Province",Mailu area,Fishing,"male, from Laluoro",M,NaN,"FATAL, multiple injuries",Y,NaN,NaN,"A. Bleakley; A.M. Rapson, p.149",1957.00.00.b-Mailu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.b,1957.00.00.b,2084.0,NaN,NaN
4220,19570000c0,1957,1957.0,Unprovoked,IRAN,Karun River,Hesamabad,NaN,Mr. Falah,M,NaN,Survived,N,NaN,Bull shark,B. Coad & F. Papahn,1957.00.00.c-Falah.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.c,1957.00.00.c,2083.0,NaN,NaN


In [646]:
#sharks['Date'].fillna('0',inplace=True)

In [647]:
def Contar_nulos(sharks):
 nan_cols = sharks.isna().sum()

 display(nan_cols[nan_cols>0])
 

In [648]:
sharks['Date'].fillna('xxxx',inplace=True) #relleno los pocos nulos que hay de esta forma para mantener el formato luego
sharks['Date'] = sharks['Date'].astype(str) #cambio todos los valores a string para trabajar con ellos

In [649]:
def Fix_s(x):

  '''
  Función que quita la s a los años que quieren representar una decada
  '''
  p = ''   
  if 's' in x:
         
         p = x[:-1]
         return p
  return x
def Fix_reported(x):

  '''
  Función que quita reported
  '''
  p = ''   
  if 'Reported' in x:
         
         p = x.split()
         try:
              
              return p[1]
         except:
              return 'No date'
              
         
  return x
sharks['Date'] = sharks['Date'].apply(Fix_reported)
sharks['Date'] = sharks['Date'].apply(Fix_s)


In [650]:

sharks = sharks[~sharks['Date'].str.contains("No date")]
sharks = sharks[~sharks['Date'].str.contains("Before")]

sharks['Date'].value_counts()


#Plantee si borrar aquellos datos que fueran solo el año pero quedan demasiados pocos datos





1957           11
1942            9
xxxx            9
1950            9
1960            8
               ..
13-Dec-1994     1
18-Dec-1994     1
30-Dec-1994     1
02-Jan-1995     1
19-Sep-1976     1
Name: Date, Length: 5320, dtype: int64

In [651]:

def Fix_format_date(x):

  '''
  Función que rellenará de xx los valores de mes y dia desconocidos de modo que todas las fechas tengan el mismo
  formato
  '''   
  if '-' in x:
       try:  
          int(x[0])
       except:
          x = 'xx-'+x
  else:
     x = 'xx-xx-'+x 
  return x
sharks['Date'] = sharks['Date'].apply(Fix_format_date)
sharks['Date'].value_counts()

xx-xx-1957     11
xx-xx-1942      9
xx-xx-xxxx      9
xx-xx-1950      9
xx-xx-1960      8
               ..
13-Dec-1994     1
18-Dec-1994     1
30-Dec-1994     1
02-Jan-1995     1
19-Sep-1976     1
Name: Date, Length: 5320, dtype: int64

In [652]:
#De la columna anterior voy a aprovechar las columnas que tengo vacías al final del dataframe para separar
#para separar año (columna que ya existe), mes y día
nombres_col[22] = 'Month'
nombres_col[23] = 'Day'
sharks.columns = nombres_col
sharks.head()

,Attack ID,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Month,Day
0,201806250,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,201806180,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,201806090,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,201806080,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,201806040,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [653]:
for i in range(len(sharks['Month'])):

  '''
  Bucle que rellena la columna Month
  '''
  sp = sharks['Date'].iloc[i].split('-')
  sharks['Month'].iloc[i] = sp[1]

for i in range(len(sharks['Day'])):

  '''
  Bucle que rellena la columna Day
  '''
  sp = sharks['Date'].iloc[i].split('-')
  sharks['Day'].iloc[i] = sp[0]

sharks.head(10)
  
              
         
 

C:\Users\pacot\AppData\Local\Temp\ipykernel_14504\2969139271.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Month'].iloc[i] = sp[1]
C:\Users\pacot\AppData\Local\Temp\ipykernel_14504\2969139271.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Day'].iloc[i] = sp[0]


,Attack ID,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Month,Day
0,201806250,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,Jun,25
1,201806180,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,Jun,18
2,201806090,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,Jun,09
3,201806080,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,Jun,08
4,201806040,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,Jun,04
5,201806030,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,Jun,03
6,201806031,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0,Jun,03
7,201805270,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0,May,27
8,201805260,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,May,26
9,201805261,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,May,26


In [654]:
sharks['Year'].fillna(0,inplace=True)
sharks['Year'] = sharks['Year'].astype(int)
sharks['Year'] = sharks['Year'].replace(0,'xxxx') 


In [655]:
#Reordeno las columnas para que lo relativo a la fecha quede junto
sharks = sharks.reindex(columns= ['Attack ID', 'Date', 'Year', 'Month', 'Day', 'Time', 'Country', 'Area', 'Location', 'Type', 'Activity', 'Name', 'Sex ','Age', 'Injury', 'Fatal (Y/N)',	'Species ','Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1',	'Case Number.2', 'original order'])
nombres_col = ['Attack ID', 'Date', 'Year', 'Month', 'Day', 'Time', 'Country', 'Area', 'Location', 'Type', 'Activity', 'Name', 'Sex','Age', 'Injury', 'Fatal (Y/N)',	'Species','Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1',	'Case Number.2', 'original order']
sharks.head()

,Attack ID,Date,Year,Month,Day,Time,Country,Area,Location,Type,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,201806250,25-Jun-2018,2018,Jun,25,18h00,USA,California,"Oceanside, San Diego County",Boating,Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,201806180,18-Jun-2018,2018,Jun,18,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",Unprovoked,Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,201806090,09-Jun-2018,2018,Jun,09,07h45,USA,Hawaii,"Habush, Oahu",Invalid,Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,201806080,08-Jun-2018,2018,Jun,08,NaN,AUSTRALIA,New South Wales,Arrawarra Headland,Unprovoked,Surfing,male,M,NaN,Minor injury to lower leg,N,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,201806040,04-Jun-2018,2018,Jun,04,NaN,MEXICO,Colima,La Ticla,Provoked,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0


In [656]:
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace('UNKNOWN',np.nan)
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace('2017',np.nan)
#Tranformo a nulos los valores que tienen "valor" pero que realmente no lo tienen

In [657]:
#Arreglo los valores nulos de la fatalidad de los ataques en base a la estadistica encontrada
#Al investigar en internet 


valores_reemplazo = ['Y', 'N']

pesos = [0.2,0.8]

valores_aleatorios = rm.choices(valores_reemplazo, pesos, k=sharks['Fatal (Y/N)'].isnull().sum())

sharks.loc[sharks['Fatal (Y/N)'].isnull(), 'Fatal (Y/N)'] = valores_aleatorios

In [658]:
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace('y','Y')
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace(' N','N')
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace('M','N')
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace('N ','N')
#Les doy el valor correcto a los valores que no lo tenían
sharks['Fatal (Y/N)'].value_counts()  

N    4734
Y    1483
Name: Fatal (Y/N), dtype: int64

In [659]:
Contar_nulos(sharks)

Time                      3272
Country                     57
Area                       443
Location                   523
Type                        13
Activity                   539
Name                       215
Sex                        568
Age                       2752
Injury                      36
Species                   2779
Investigator or Source      26
pdf                          9
href formula                10
href                         9
Case Number.1                9
Case Number.2                9
original order               2
dtype: int64

In [660]:
sharks['Time'].value_counts()

Afternoon      187
11h00          128
Morning        120
12h00          109
15h00          108
              ... 
00h30            1
14h37            1
10h07            1
13h53            1
19h00-20h00      1
Name: Time, Length: 365, dtype: int64

In [661]:
sharks['Time'] = sharks['Time'].replace(['Dusk','dusk','Dawn','Early morning','Early Morning','Morning','Mid-morning','Late morning','Noon','Midday','After noon','Afternoon','Late afternoon','Late Afternoon','Evening','Night','Midnight','Late night','2 hrs before sunset'],['03h00','03h00','05h00','04h00','04h00','08h00','10h00','11h00','12h00','12h00','15h00','15h00','17h00','17h00','20h00','22h00','00h00','02h00','22h00'])
sharks['Time'] = sharks['Time'].replace('--',np.nan)
sharks['Time'] = sharks['Time'].replace('11h01 -time of ship sinking',np.nan)
sharks['Time'] = sharks['Time'].replace('Ship aban-doned at 03h10',np.nan)
#elimino todos los tiempos aproximados por palabra, igualo "--" a nulo que viene a ser lo mismo y corrijo dos casos especificos

In [662]:
sharks['Time'].fillna('xxhxx',inplace=True)

In [663]:
filtrado = sharks.loc[sharks['Time'].str.contains('-')] #Miro los casos de horas por rangos
print (filtrado)

       Attack ID         Date  Year Month Day            Time  \
1      201806180  18-Jun-2018  2018   Jun  18   14h00  -15h00   
356    201509240  24-Sep-2015  2015   Sep  24     10h45-11h15   
536    201407051   5-Jul-2014  2014   Jul   5   07h00 - 08h00   
537    201407030   3-Jul-2014  2014   Jul   3     18h15-18h30   
704    201302090  09-Feb-2013  2013   Feb  09   09h00 - 09h30   
1193   200811060  06-Nov-2008  2008   Nov  06  10h00 -- 11h00   
1881   200202160  16-Feb-2002  2002   Feb  16    09h00 -10h00   
3801   196208120  12-Aug-1962  1962   Aug  12   14h00 - 15h00   
3823   196205120  12-May-1962  1962   May  12   03h45 - 04h00   
3998   196004140  14-Apr-1960  1960   Apr  14   09h00 - 09h30   
5963   186212220  22-Dec-1862  1862   Dec  22  06h00 -- 07h00   
6091   183004300  30-Apr-1830  1830   Apr  30     17h00-18h00   
6105  18260520R0  20-May-1826  1826   May  20     19h00-20h00   

               Country             Area  \
1                  USA          Georgia   
356

In [664]:
sharks['Time'] = sharks['Time'].replace('14h00  -15h00','14h30')
def Fix_ranges(x):

  '''
  Función que eliminará los rangos de las horas 
  '''
  p = ''
  if '--'in x:
       p = x.split('--')
       l = p[0].split('h')
       x = l[0]+'h'+'30'
       return x
          
  if '-' in x:
         
         p = x.split('-')
         print(p)
         if p[0][-2] == '0':
               if '3' in p[1]:
                    l = p[0].split('h')
                    x = l[0]+'h'+'15'
               elif p[1][-2] == '0':
                    l = p[0].split('h')
                    x = l[0]+'h'+'30'
         elif p[0][-2] == '4':
              if p[1][-2] == '1':
                   x = '11h00'
              else:
                   x = '03h52'
         elif p[0][-2] == '1':
              x = '18h22'
   # Convertir strings completamente en palabras a 'Unknown'
  if not re.match(r'^\d+|\d+.\d+$', x):
        
        return 'xxhxx'

    # Reemplazar comillas, signos de interrogación por nada
  x = re.sub(r'["?]', '', x)

    # Reemplazar letras por nada
  if (x[2] != 'h'):
     x = re.sub(r'[a-zA-Z]', '', x)
     if len(x) == 4:
          f = list(x)
          x = f[0]+f[1]+'h'+f[2]+f[3]
  if len(x) == 6:
       x = x[:-1]
  if x[2] == ':':
     f = x.split(':')
     x = f[0]+'h'+f[1]
  j = x.split()
  if 'Sunset' in x:
       u = list(x)
       x = u[0]+u[1]+u[2]+u[3]+u[4]
  try:
     if j[1] == '/' or j[1] == '&' or j[1] == 'to' or j[1] == 'or':
          x = j[0]
  except:
       return x      
  return x

sharks['Time'] = sharks['Time'].apply(Fix_ranges)
#Casos especiales

sharks['Time'] = sharks['Time'].replace('03h45 - 04h00','03h52')


['10h45', '11h15']
['07h00 ', ' 08h00']
['18h15', '18h30']
['09h00 ', ' 09h30']
['09h00 ', '10h00']
['14h00 ', ' 15h00']
['03h45 ', ' 04h00']
['09h00 ', ' 09h30']
['17h00', '18h00']
['19h00', '20h00']


In [665]:
#Renombro la columna a hora para que quede más claro
nombres_col[5] = 'Hour'
sharks.columns = nombres_col
sharks.head()

,Attack ID,Date,Year,Month,Day,Hour,Country,Area,Location,Type,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,201806250,25-Jun-2018,2018,Jun,25,18h00,USA,California,"Oceanside, San Diego County",Boating,Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,201806180,18-Jun-2018,2018,Jun,18,14h30,USA,Georgia,"St. Simon Island, Glynn County",Unprovoked,Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,201806090,09-Jun-2018,2018,Jun,09,07h45,USA,Hawaii,"Habush, Oahu",Invalid,Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,201806080,08-Jun-2018,2018,Jun,08,xxhxx,AUSTRALIA,New South Wales,Arrawarra Headland,Unprovoked,Surfing,male,M,NaN,Minor injury to lower leg,N,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,201806040,04-Jun-2018,2018,Jun,04,xxhxx,MEXICO,Colima,La Ticla,Provoked,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0


In [666]:

sharks['Country'].value_counts()

USA                       2215
AUSTRALIA                 1323
SOUTH AFRICA               571
PAPUA NEW GUINEA           132
NEW ZEALAND                127
                          ... 
NORTH SEA                    1
RED SEA / INDIAN OCEAN       1
BRITISH WEST INDIES          1
ADMIRALTY ISLANDS            1
CEYLON (SRI LANKA)           1
Name: Country, Length: 207, dtype: int64

In [667]:
for i in range(len(sharks['Country'])):

  '''
  Bucle que rellena los nulos del pais/región si el area tiene dato
  '''
  if sharks['Country'].iloc[i] is np.nan:
     sharks['Country'].iloc[i] = sharks['Area'].iloc[i]

#Algunos datos especificos
sharks['Country'].iloc[4639] = 'Hong Kong'
sharks['Area'].iloc[4639] = 'Hong Kong'
sharks['Area'].iloc[3605] = 'Florida Strait'
sharks['Country'].iloc[6155] = 'Barbados'
sharks['Area'].iloc[6155] = 'Carlisle Bay'
sharks['Country'].iloc[5810] = 'Argentina'
sharks['Area'].iloc[5810] = 'Santa Cruz'


#sharks.loc[sharks['Country'].isna()]


C:\Users\pacot\AppData\Local\Temp\ipykernel_14504\2619594468.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Country'].iloc[i] = sharks['Area'].iloc[i]
C:\Users\pacot\AppData\Local\Temp\ipykernel_14504\2619594468.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Country'].iloc[4639] = 'Hong Kong'
C:\Users\pacot\AppData\Local\Temp\ipykernel_14504\2619594468.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shar

In [668]:
 #Tras una investigación determino que USA es el lugar donde es más común que sucedan los ataques
 #Además coincide con la moda con lo cual relleno los nulos con este dato

sharks['Country'].fillna('USA',inplace=True)
sharks['Country'] = sharks['Country'].str.title()
sharks['Country'].value_counts()

Usa                   2255
Australia             1323
South Africa           571
Papua New Guinea       132
New Zealand            127
                      ... 
Grand Cayman             1
Mexico                   1
Sudan?                   1
The Balkans              1
Ceylon (Sri Lanka)       1
Name: Country, Length: 214, dtype: int64

In [669]:
filtrado = sharks.loc[sharks['Country'].str.contains('/')] #Miro en cuantos duda entre paises
print (filtrado)

       Attack ID         Date  Year Month Day   Hour  \
2292   199607231  23-Jul-1996  1996   Jul  23  18h00   
3957  19601000d0  xx-Oct-1960  1960   Oct  xx  xxhxx   
4140   195809130  13-Sep-1958  1958   Sep  13  14h00   
4619   194508190  19-Aug-1945  1945   Aug  19  xxhxx   
4696   194211130  13-Nov-1942  1942   Nov  13  xxhxx   
4934  19340826R0  26-Aug-1934  1934   Aug  26  xxhxx   
5978  18610212R0  12-Feb-1861  1861   Feb  12  xxhxx   

                           Country                               Area  \
2292                Egypt / Israel         South Sinai, Gulf of Aqaba   
3957        Red Sea / Indian Ocean  Enroute from Suez to Aden (Yemen)   
4140    Andaman / Nicobar Islandas                                NaN   
4619                   Iran / Iraq                Shatt-al-Arab River   
4696     Solomon Islands / Vanuatu                                NaN   
4934               Italy / Croatia                                NaN   
5978  Equatorial Guinea / Cameroon      

In [670]:
#Me quedaré con el primer pais/región por ser ocasionalmente más restrictivo

def Fix_region(x):

  '''
  Función que eliminará las dudas entre paises divididos por /
  '''
  p = ''  
  if '/' in x:
         
         p = x.split('/')
         x = p[0]
              
         
  return x

sharks['Country'] = sharks['Country'].apply(Fix_region)
sharks['Country'] = sharks['Country'].replace('Usa','USA')
sharks['Country'] = sharks['Country'].replace('Sudan?','Sudan')

filtrado = sharks.loc[sharks['Country'].str.contains('/')] #Miro en cuantos duda entre paises
print (filtrado)

Empty DataFrame
Columns: [Attack ID, Date, Year, Month, Day, Hour, Country, Area, Location, Type, Activity, Name, Sex, Age, Injury, Fatal (Y/N), Species, Investigator or Source, pdf, href formula, href, Case Number.1, Case Number.2, original order]
Index: []


In [671]:
def Fix_parentesis(x):
    p = ''
    if '(' in x:
        p = x.split('(')
        x = p[1][:-1]
    
    
    return x
sharks['Country'] = sharks['Country'].apply(Fix_parentesis)

In [672]:
#Finalmente renombro la columna a Country/Region ya que no siempre es un pais el dato que aparece
#Sin ir más lejos puede ser en mar abierto
nombres_col[6] = 'Country/Region'
sharks.columns = nombres_col
sharks.head()

,Attack ID,Date,Year,Month,Day,Hour,Country/Region,Area,Location,Type,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,201806250,25-Jun-2018,2018,Jun,25,18h00,USA,California,"Oceanside, San Diego County",Boating,Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,201806180,18-Jun-2018,2018,Jun,18,14h30,USA,Georgia,"St. Simon Island, Glynn County",Unprovoked,Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,201806090,09-Jun-2018,2018,Jun,09,07h45,USA,Hawaii,"Habush, Oahu",Invalid,Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,201806080,08-Jun-2018,2018,Jun,08,xxhxx,Australia,New South Wales,Arrawarra Headland,Unprovoked,Surfing,male,M,NaN,Minor injury to lower leg,N,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,201806040,04-Jun-2018,2018,Jun,04,xxhxx,Mexico,Colima,La Ticla,Provoked,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0


In [673]:
#Rellenamos los nulos 
for i in range(len(sharks['Area'])):
  '''
  Bucle que rellena los nulos del Area si el Location tiene dato
  Prefiero tener el dato de la localización como area (que en muchos casos es un dato representativo)
  Antes de poner "desconocido"
  '''
  if sharks['Area'].iloc[i] is np.nan:
     sharks['Area'].iloc[i] = sharks['Location'].iloc[i]

sharks['Area'].fillna('Unknown',inplace=True)
sharks['Location'].fillna('Unknown',inplace=True)

C:\Users\pacot\AppData\Local\Temp\ipykernel_14504\562506109.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Area'].iloc[i] = sharks['Location'].iloc[i]


In [674]:
Contar_nulos(sharks)

Type                        13
Activity                   539
Name                       215
Sex                        568
Age                       2752
Injury                      36
Species                   2779
Investigator or Source      26
pdf                          9
href formula                10
href                         9
Case Number.1                9
Case Number.2                9
original order               2
dtype: int64

In [675]:
sharks.loc[sharks['Location']=='Carlisle Bay']

,Attack ID,Date,Year,Month,Day,Hour,Country/Region,Area,Location,Type,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
6155,174212170,17-Dec-1742,1742,Dec,17,xxhxx,USA,Carlisle Bay,Carlisle Bay,Unprovoked,Swimming,2 impressed seamen,M,NaN,FATAL,Y,NaN,"C. Moore, GSAF",1742.12.17-AdviceSeamen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1742.12.17,1742.12.17,148.0
6160,170303260,26-Mar-1703,1703,Mar,26,22h00,Barbados,Southwest coast,Carlisle Bay,Unprovoked,Swimming,"Samuel Jennings, a deserter from the British f...",M,19,"Hand and foot severely bitten, surgically ampu...",N,NaN,"W.R.Cutter, Vol.1, p.252",1703.03.26-Jennings.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1703.03.26,1703.03.26,143.0


In [676]:
pd.unique(sharks['Hour'])

array(['18h00', '14h30', '07h45', 'xxhxx', '17h00', '14h00', '08h00',
       '15h00', '08h15', '11h00', '10h30', '10h40', '16h50', '07h00',
       '09h30', '21h50', '09h40', '17h35', '15h30', '07h30',
       '19h00, Dusk', '22h00', '16h00', '15h01', '12h00', '13h45',
       '23h30', '09h00', '18h30', '12h30', '16h30', '18h45', '06h00',
       '10h00', '10h44', '13h19', '13h30', '10h45', '11h20', '11h45',
       '19h30', '08h30', '15h45', '17h34', '17h10', '11h15', '08h50',
       '17h45', '13h00', '10h20', '13h20', '02h00', '09h50', '11h30',
       '17h30', '900', '10h43', '15h15', '15h40', '19h05', '16h20',
       '14h34', '15h25', '14h55', '17h46', '15h49', '19h00', '00h00',
       '10h15', '18h15', '04h00', '14h50', '13h50', '19h20', '10h25',
       '16h45', '15h52', '06h15', '14h40', '19h45', '10h10', '08h45',
       '20h00', '12h10', '18h05', '11h41', '12h25', '17h51', '16h12',
       '12h45', '10h50', '14h15', '09h45', '06h45', '05h00', '03h30',
       '06h30', '17h20', '16h40', 